In [1]:
import torch
import torch.nn.functional as F

torch.manual_seed(1)

## <strong> 1. 파이토치로 소프트맥스의 비용 함수 구현하기 (로우-레벨) </strong>

소프트맥스 회귀를 구현하기 전에 비용 함수를 로우-레벨로 구현해보자.

In [2]:
z = torch.FloatTensor([1, 2, 3])


이 텐서를 소프트맥스 함수에 입력해보자.

In [4]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)
print(hypothesis.sum())

tensor([0.0900, 0.2447, 0.6652])
tensor(1.)


위처럼 총 원소의 값의 합은 1이 된다. <br>
이번에는 비용 함수를 직접 구현해보자. <br>
먼저 $3 \times 5$ 크기의 행렬을 소프트맥스 함수에 통과시켜보자.

In [8]:
z = torch.rand((3, 5), requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2570, 0.1524, 0.2303, 0.1721, 0.1882],
        [0.1178, 0.1501, 0.2499, 0.1687, 0.3135],
        [0.2615, 0.1711, 0.1254, 0.2375, 0.2045]], grad_fn=<SoftmaxBackward0>)


각 셈플의 레이블을 선언하고 원-핫 인코딩을 수행해보자.

In [14]:
y = torch.randint(5, (3,))
print(y)

tensor([3, 0, 4])


In [15]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])

위 코드에서 unsqueeze의 경우 해당 벡터의 주어진 차원을 늘려준다. <br>
즉, y의 크기가 $3 \times 1$이 된다.

In [16]:
print(y.unsqueeze(1))

tensor([[3],
        [0],
        [4]])


**scatter 함수**는 지정한 차원을 0~n까지 차례로 이동하면서 주어진 인덱스에 맞게 주어진 값을 채운다.

In [17]:
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])

In [21]:
src = torch.arange(1, 11).reshape((2, 5))
index = torch.tensor([[0, 1, 2, 0]])
torch.zeros(3, 5, dtype=src.dtype).scatter_(0, index, src)

tensor([[1, 0, 0, 4, 0],
        [0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0]])

$cost(W) = -\frac{1}{n}\sum_{i=1}^{n}\sum_{j=1}^{k}{y_j^{(i)}log(p_j^{(i)})}$

해당 식을 코드로 구현하면 아래와 같다.

In [22]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.8286, grad_fn=<MeanBackward0>)


## <strong> 2. 파이토치로 소프트맥스의 비용 함수 구현하기 (하이-레벨) </strong>

<h3> <strong> 2.1 F.softmax() + torch.log() = F.log_softmax() </strong> </h3>

앞에서 소프트맥스 함수의 결과에 로그를 씌울 때는 소프트맥스 함수의 출력값에 로그 함수를 적용했다. <br>
이때 파이토치의 **F.log_softmax()**를 활용하면 이를 한 번에 해결할 수 있다. 

In [23]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3585, -1.8809, -1.4685, -1.7599, -1.6704],
        [-2.1387, -1.8964, -1.3867, -1.7796, -1.1600],
        [-1.3414, -1.7656, -2.0759, -1.4377, -1.5870]], grad_fn=<LogBackward0>)

In [24]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.3585, -1.8809, -1.4685, -1.7599, -1.6704],
        [-2.1387, -1.8964, -1.3867, -1.7796, -1.1600],
        [-1.3414, -1.7656, -2.0759, -1.4377, -1.5870]],
       grad_fn=<LogSoftmaxBackward0>)

<h3> <strong> 2.2 F.log_softmax() + F.nll_loss() = F.cross_entropy() </strong> </h3>

이전에 구현한 비용 함수는 다음과 같다.

In [25]:
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.8286, grad_fn=<MeanBackward0>)

**F.log_softmax()**를 활용하면 다음과 같이 사용할 수 있다.

In [27]:
(y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.8286, grad_fn=<MeanBackward0>)

**F.nll_loss()**를 사용하면 원-핫 벡터를 넣을 필요없이 실제값을 인자로 사용할 수 있다.

In [29]:
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.8286, grad_fn=<NllLossBackward0>)

**F.cross_entropy()**를 사용하면 해당 비용 함수의 기능을 한 번에 수행해준다.

In [30]:
F.cross_entropy(z, y)

tensor(1.8286, grad_fn=<NllLossBackward0>)

출처: https://wikidocs.net/60572